# TensorFlow 2.0 alpha - Custom Layers

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Layers
#### For many Machine Learning operations higher level abstraction is needed - Not specification of individual variables/operations
#### TF provides many simple, common layers capable of ML operations - As well as the ability to write new, or mixed-existing layers

### Layers are Objects

In [4]:
# First argument for most, is # of output dimensions

layer = tf.keras.layers.Dense(100)

# Input argument, often implied, can be used as well

layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

#### There are many pre-existing Layers - including Dense (fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, etc.
### To use Layer, Call it

In [5]:
layer(tf.zeros([10, 5]))

<tf.Tensor: id=59, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Inspect layer Variables with layer.variable - and trainable variables, with layer.trainable_variables
#### Fully-connected layers - variables for weights and biases

In [6]:
layer.variables

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.25257814,  0.01163477, -0.24498141,  0.09267968,  0.41632444,
          0.32417148, -0.21949351,  0.14460891, -0.2771877 ,  0.11145324],
        [ 0.5078184 ,  0.52482957, -0.11163193, -0.12754756,  0.02461725,
          0.09317684,  0.1122179 , -0.02014148, -0.27136603,  0.08979642],
        [-0.19607595,  0.06553143, -0.25996608, -0.40616053,  0.12482899,
          0.00111449,  0.27387106,  0.23684663, -0.46238   , -0.20166436],
        [-0.02108878,  0.4665267 , -0.615757  ,  0.48804277, -0.18181401,
          0.22222596, -0.5824282 ,  0.60214454,  0.2684703 ,  0.603854  ],
        [-0.59859943,  0.23128837, -0.52116936,  0.46380764,  0.15718937,
         -0.28825155, -0.16088012, -0.12785834, -0.40382934, -0.39580446]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

#### Variables are also accessible through accessors

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.25257814,  0.01163477, -0.24498141,  0.09267968,  0.41632444,
          0.32417148, -0.21949351,  0.14460891, -0.2771877 ,  0.11145324],
        [ 0.5078184 ,  0.52482957, -0.11163193, -0.12754756,  0.02461725,
          0.09317684,  0.1122179 , -0.02014148, -0.27136603,  0.08979642],
        [-0.19607595,  0.06553143, -0.25996608, -0.40616053,  0.12482899,
          0.00111449,  0.27387106,  0.23684663, -0.46238   , -0.20166436],
        [-0.02108878,  0.4665267 , -0.615757  ,  0.48804277, -0.18181401,
          0.22222596, -0.5824282 ,  0.60214454,  0.2684703 ,  0.603854  ],
        [-0.59859943,  0.23128837, -0.52116936,  0.46380764,  0.15718937,
         -0.28825155, -0.16088012, -0.12785834, -0.40382934, -0.39580446]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

# Implementing Custom Layers
### Extend tf.keras.layers class, implementing:
#### __init__ - can do all input-independent initialization - required Shapes for variables must be specified
#### __build__ - when creating variables after learning the input Shapes - *Late* variable creation

In [8]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_variable('kernel', shape=[int(input_shape[-1]),
                                                        self.num_outputs])
    
    def call(self, input):
        return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

print(layer(tf.zeros([10,5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.31676382, -0.35606778, -0.55156136,  0.48472935, -0.26616472,
         0.56535727,  0.34881133, -0.03091329,  0.5743274 , -0.3905552 ],
       [-0.32543886, -0.58241904,  0.25177127,  0.16202146,  0.518887  ,
        -0.25680554, -0.17120555, -0.5065768 , -0.12480593, -0.2116181 ],
       [-0.4962682 ,  0.5740709 , -0.41251507,  0.1451804 ,  0.5029914 ,
         0.09180254, -0.3872588 , -0.23413709, -0.6006301 , -0.36873376],
       [-0.02486002,  0.34510267, -0.4848554 , -0.03226358,  0.18390155,
         0.62642556, -0.17898566, -0

#### call - does the forward computation
# Models - Composing Layers
#### Use tf.keras.Model - to create a layer-like container of other layers
#### example - resnet, with each residual block containing convolutions, batch normalizations, and a shortcut

In [9]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])

print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization_v2/gamma:0', 'resnet_identity_block/batch_normalization_v2/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_v2_1/gamma:0', 'resnet_identity_block/batch_normalization_v2_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_v2_2/gamma:0', 'resnet_identity_block/batch_normalization_v2_2/beta:0']


#### Use tf.keras.Sequential - for other models with layers, where simply one is called after another

In [12]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1,1), input_shape=(None,None,3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1, padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=756, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

# Conclusion
### These methods will allow for adapting models, using custom layers, or models, for better structure